# train mlp on MNIST using fp32

In [ ]:
import numpy as np
import tensorflow as tf
from dataset import load_mnist
from tensorflow import keras
from keras import layers

# load dataset
x_train, y_train, x_test, y_test = load_mnist()

In [ ]:
x = x_in = keras.layers.Input((28,28,1))

x = keras.layers.Conv2D(16, kernel_size=3, padding="SAME")(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.Conv2D(32, kernel_size=3, padding="SAME")(x)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D()(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256)(x)
x = keras.layers.ReLU()(x)
x = keras.layers.Dense(256)(x)
x = keras.layers.ReLU()(x)
x = keras.layers.Dense(10)(x)

lenet = keras.Model(inputs=[x_in], outputs=[x])
lenet.summary()
lenet.compile(optimizer=keras.optimizers.Adam(0.001), loss=keras.losses.CategoricalCrossentropy(from_logits=True, label_smoothing=0.02), metrics=["accuracy"])
hist = lenet.fit(x_train, y_train, 256, 10, validation_data=(x_test, y_test))


In [ ]:
from NeuralNetwork import QNeuralNetworkWithScale, QLeNet
import Activations
import FullyConnectedLayer 


input_shape = x_train.shape[1:]

# Create and train the neural network
qlenet = QLeNet(input_shape=input_shape, output_size=y_train.shape[-1], batch_size=256)

In [ ]:
# como vincular um modelo com o outro?
# mais vale criar um modelo do 0 com base no modelo q vier...
# o problema maior é criar cada camada
# a rede é só um array de camadas

import matplotlib.pyplot as plt
import quantizer

from ConvLayer import *

dn_layers = []
for l in lenet.layers:
    if isinstance(l, keras.layers.Conv2D):    
        print("instanciating conv layer...")
        l.weights[0].shape[0],l.weights[0].shape[1]

        w_shape = l.weights[0].shape
        nfilters = w_shape[3]
        kernel_size = w_shape[0]
        input_channels = w_shape[2]
        strides=[1,1,1,1] ### TODO: variable strides
        padding= l.padding

        # create QCONVLAYER
        qfc = QConvLayer(nfilters, kernel_size, input_channels, strides, padding)
        
        fpw = l.weights[0].numpy()        
        fpb  = l.weights[1].numpy()
        
        w_scale = np.max(np.abs(fpw))
        
        fpw_scaled = fpw / w_scale
        qw = quantizer.quantize(fpw_scaled, True, False)
        
        # atribui o peso quantizado
        qfc.qw = qw
        qfc.weights_scale = fpw_scaled
               

        plt.hist(np.ravel(qw), bins=64)
        plt.hist(np.ravel(fpw_scaled), bins=64)
        plt.hist(np.ravel(fpw), bins=64)
        plt.show()

        
        fpb_scaled = fpb / w_scale
        qb = quantizer.quantize(fpb_scaled, True, False)
        qfc.qb = qb
        plt.hist(np.ravel(qb), bins=64)
        plt.hist(np.ravel(fpb_scaled), bins=64)
        plt.hist(np.ravel(fpb), bins=64)
        plt.show()


        dn_layers.append(qfc)

    if isinstance(l, keras.layers.MaxPool2D):    
        print(l)
        dn_maxpool = CustomMaxPool(l.pool_size, l.strides)
        dn_layers.append(dn_maxpool)

    if isinstance(l, keras.layers.Flatten):    
        dn_layers.append(CustomFlatten(l.input_shape))
    if isinstance(l, keras.layers.Dense):        
        
        qfc = FullyConnectedLayer.QFullyConnectedLayerWithScale(l.weights[0].shape[0],l.weights[0].shape[1])
        
        fpw = l.weights[0].numpy()        
        fpb  = l.weights[1].numpy()
        
        w_scale = np.max(np.abs(fpw))
        
        fpw_scaled = fpw / w_scale
        qw = quantizer.quantize(fpw_scaled, True, False)
        
        # atribui o peso quantizado
        qfc.qw = qw
        qfc.weights_scale = fpw_scaled
               

        plt.hist(np.ravel(qw), bins=64)
        plt.hist(np.ravel(fpw_scaled), bins=64)
        plt.hist(np.ravel(fpw), bins=64)
        plt.show()

        
        fpb_scaled = fpb / w_scale
        qb = quantizer.quantize(fpb_scaled, True, False)
        qfc.qb = qb
        plt.hist(np.ravel(qb), bins=64)
        plt.hist(np.ravel(fpb_scaled), bins=64)
        plt.hist(np.ravel(fpb), bins=64)
        plt.show()


        dn_layers.append(qfc)


    if isinstance(l, keras.layers.ReLU):                
        dn_layers.append(Activations.QReLU())

print(dn_layers)




In [ ]:
qlenet.load_layers_from_model(lenet)

In [ ]:
y_pred = qlenet.predict(x_test, 256)
print(y_pred.shape)
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(y_pred == tf.argmax(y_test, axis=1), tf.float32))
print(f"Accuracy: {accuracy * 100}%")

In [ ]:
# finetune the dnn
qlenet.train(x_train, y_train, learning_rate=0.000010, num_epochs=1, x_val=x_test, y_val=y_test)
